# Getting started with PPO and ProcGen

Here's a bit of code that should help you get started on your projects.

The cell below installs `procgen` and downloads a small `utils.py` script that contains some utility functions. You may want to inspect the file for more details.

In [1]:
!pip install procgen
!wget https://raw.githubusercontent.com/nicklashansen/ppo-procgen-utils/main/utils.py

     |████████████████████████████████| 39.9MB 82kB/s 
     |████████████████████████████████| 51kB 9.0MB/s 
     |████████████████████████████████| 665kB 56.1MB/s 
     |████████████████████████████████| 3.3MB 52.7MB/s 
     |████████████████████████████████| 204kB 62.2MB/s 
     |████████████████████████████████| 22.2MB 36.6MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: imageio 2.4.1
    Uninstalling imageio-2.4.1:
      Successfully uninstalled imageio-2.4.1
--2021-01-02 02:51:38--  https://raw.githubusercontent.com/nicklashansen/ppo-procgen-utils/main/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14807 (14K) [text/plain]
Saving to: ‘utils.py

Hyperparameters. These values should be a good starting point. You can modify them later once you have a working implementation.

In [2]:
# Hyperparameters
total_steps = 10e6
num_envs = 32
num_levels = 100
num_steps = 256
num_epochs = 3
batch_size = 512
eps = .2
grad_eps = .5
value_coef = .5
entropy_coef = .01 #.01

#####################################
n_stack = 3 # =1 for default
#####################################

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from utils import *# make_env, Storage, orthogonal_init
from google.colab import files

###############################################################################  
def make_env(
	n_envs=32,
	env_name='starpilot',
	start_level=0,
	num_levels=100,
	use_backgrounds=True,
	normalize_obs=False,
	normalize_reward=True,
	seed=0,

	):
	"""Make environment for procgen experiments"""
	set_global_seeds(seed)
	set_global_log_levels(40)
	env = ProcgenEnv(
		num_envs=n_envs,
		env_name=env_name,
		start_level=start_level,
		num_levels=num_levels,
		distribution_mode='easy',
		use_backgrounds=use_backgrounds,
		restrict_themes=not use_backgrounds,
		render_mode='rgb_array',
		rand_seed=seed
	)
    
  
	env = VecExtractDictObs(env, "rgb")
	if n_stack >=2 :
		env = VecFrameStack(env, n_stack)
	env = VecNormalize(env, ob=normalize_obs, ret=normalize_reward)
	env = TransposeFrame(env)
	env = ScaledFloatFrame(env)
	env = TensorEnv(env)
	

	return env
###############################################################################


def xavier_uniform_init(module, gain=1.0):
    if isinstance(module, nn.Linear) or isinstance(module, nn.Conv2d):
        nn.init.xavier_uniform_(module.weight.data, gain)
        nn.init.constant_(module.bias.data, 0)
    return module



class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.size(0), -1)



class ResidualBlock(nn.Module):
    def __init__(self,
                 in_channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=in_channels, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=in_channels, out_channels=in_channels, kernel_size=3, stride=1, padding=1)

    def forward(self, x):
        out = nn.ReLU()(x)
        out = self.conv1(out)
        out = nn.ReLU()(out)
        out = self.conv2(out)
        return out + x

class ImpalaBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ImpalaBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1)
        self.res1 = ResidualBlock(out_channels)
        self.res2 = ResidualBlock(out_channels)

    def forward(self, x):
        x = self.conv(x)
        x = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)(x)
        x = self.res1(x)
        x = self.res2(x)
        return x

class ImpalaModel(nn.Module):
    def __init__(self,
                 in_channels,
                 **kwargs):
        super(ImpalaModel, self).__init__()
        self.block1 = ImpalaBlock(in_channels=in_channels, out_channels=16)
        self.block2 = ImpalaBlock(in_channels=16, out_channels=32)
        self.block3 = ImpalaBlock(in_channels=32, out_channels=32)
        self.fc = nn.Linear(in_features=32 * 8 * 8, out_features=256)

        self.output_dim = 256
        self.apply(xavier_uniform_init)

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = nn.ReLU()(x)
        x = Flatten()(x)
        x = self.fc(x)
        x = nn.ReLU()(x)
        return x


class Encoder(nn.Module):
  def __init__(self, in_channels, feature_dim):
    super().__init__()
    self.layers = nn.Sequential(
        nn.Conv2d(in_channels=in_channels, out_channels=32, kernel_size=8, stride=4), nn.ReLU(),
        nn.Conv2d(in_channels=32, out_channels=64, kernel_size=4, stride=2), nn.ReLU(),
        nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1), nn.ReLU(),
        Flatten(),
        nn.Linear(in_features=1024, out_features=feature_dim), nn.ReLU()
    )
    self.apply(orthogonal_init)

  def forward(self, x):
    return self.layers(x)


class Policy(nn.Module):
  def __init__(self, encoder, feature_dim, num_actions):
    super().__init__()
    self.encoder = encoder
    self.policy = orthogonal_init(nn.Linear(feature_dim, num_actions), gain=.01)
    self.value = orthogonal_init(nn.Linear(feature_dim, 1), gain=1.)

  def act(self, x):
    with torch.no_grad():
      x = x.cuda().contiguous()
      dist, value = self.forward(x)
      action = dist.sample()
      log_prob = dist.log_prob(action)
    
    return action.cpu(), log_prob.cpu(), value.cpu()

  def forward(self, x):
    x = self.encoder(x)
    logits = self.policy(x)
    value = self.value(x).squeeze(1)
    dist = torch.distributions.Categorical(logits=logits)

    return dist, value

########################################################################################################################################

print("starting to make env")

# Define environmentbossfight
# check the utils.py file for info on arguments
env = make_env(num_envs, num_levels=num_levels, env_name='starpilot')
print('Observation space:', env.observation_space)
print('Action space:', env.action_space.n)

# Define network
#encoder = Encoder(3,512)
observation_shape = env.observation_space.shape
in_channels = observation_shape[0]
encoder = ImpalaModel(in_channels=in_channels)
policy = Policy(encoder, 256, 15)
policy.load_state_dict(torch.load("checkpoint_Final.pt"))
policy.cuda()
# Define optimizer
# these are reasonable values but probably not optimal
optimizer = torch.optim.Adam(policy.parameters(), lr=5e-4, eps=1e-5) #lr=5e-4 , eps=1e-5

# Define temporary storage
# we use this to collect transitions during each iteration
storage = Storage(
    env.observation_space.shape,
    num_steps,
    num_envs
)


# Run training
obs = env.reset()
step = 5005312
i = 10
print("NN setup, Training Starts")
while step < total_steps:

  # Use policy to collect data for num_steps steps
  policy.eval()
  for _ in range(num_steps):
    # Use policy
    action, log_prob, value = policy.act(obs)
    
    # Take step in environment
    next_obs, reward, done, info = env.step(action)

    # Store data
    storage.store(obs, action, reward, done, info, log_prob, value)
    
    # Update current observation
    obs = next_obs

  # Add the last observation to collected data
  _, _, value = policy.act(obs)
  storage.store_last(obs, value)

  # Compute return and advantage
  storage.compute_return_advantage()

  # Optimize policy
  policy.train()
  for epoch in range(num_epochs):

    # Iterate over batches of transitions
    generator = storage.get_generator(batch_size)
    for batch in generator:
      b_obs, b_action, b_log_prob, b_value, b_returns, b_advantage = batch

      # Get current policy outputs
      new_dist, new_value = policy(b_obs)
      new_log_prob = new_dist.log_prob(b_action)
      # log_prob
      # Clipped policy objective
      #print(str(log_prob.shape) + " " + str(b_log_prob.shape) + " " + str(new_log_prob.shape))
      ratio = torch.exp(new_log_prob - b_log_prob)
      
      clipped_ratio = ratio.clamp(min=1.0 - eps, max=1.0 + eps) 
      policy_reward = torch.min(ratio * b_advantage, clipped_ratio * b_advantage)
      #clip_fraction = (abs((ratio - 1.0)) > clip).to(torch.float).mean()
      pi_loss = -policy_reward.mean()

      # Clipped value function objective
      # clipped_value = new_value + (b_value - new_value).clamp(min=-eps,max=eps)
      # vf_loss=torch.max((b_value-b_returns)**2, (clipped_value-b_returns)**2)
      # value_loss = 0.5 * vf_loss.mean()

      clipped_value = b_value + (new_value - b_value).clamp(min=-eps,max=eps) 

      clipped_value = (new_value - b_value).clamp(min=-eps,max=eps)
      value_loss = 0.5 * torch.max(torch.pow(new_value - b_returns,2), torch.pow(clipped_value - b_returns, 2)).mean()

      # Entropy loss
      entropy_loss = new_dist.entropy().mean()

      # Backpropagate losses
      # loss = torch.mean(pi_loss+value_coef*value_loss+entropy_coef*entropy_loss) #
      loss = pi_loss + value_coef * value_loss - entropy_coef * entropy_loss
      loss.backward()

      # Clip gradients
      torch.nn.utils.clip_grad_norm_(policy.parameters(), grad_eps)

      # Update policy
      optimizer.step()
      optimizer.zero_grad()

  # Update stats
  step += num_envs * num_steps
  print(f'Step: {step}\tMean reward: {storage.get_reward()}')

  if step//503808>=i:
    torch.save(policy.state_dict(), 'checkpoint'+str(i)+'.pt')
    files.download('checkpoint'+str(i)+'.pt')
    i=i+1

print('Completed training!')
torch.save(policy.state_dict(), 'checkpoint_Final_Final.pt')


files.download('checkpoint_Final_Final.pt')

starting to make env
Observation space: Box(0.0, 1.0, (9, 64, 64), float32)
Action space: 15
NN setup, Training Starts
Step: 5013504	Mean reward: 9.5
Step: 5021696	Mean reward: 10.125
Step: 5029888	Mean reward: 10.5
Step: 5038080	Mean reward: 10.84375


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Step: 5046272	Mean reward: 11.25
Step: 5054464	Mean reward: 10.40625
Step: 5062656	Mean reward: 8.8125
Step: 5070848	Mean reward: 8.6875
Step: 5079040	Mean reward: 10.25
Step: 5087232	Mean reward: 9.5
Step: 5095424	Mean reward: 9.5625
Step: 5103616	Mean reward: 10.0
Step: 5111808	Mean reward: 10.53125
Step: 5120000	Mean reward: 9.59375
Step: 5128192	Mean reward: 9.46875
Step: 5136384	Mean reward: 9.90625
Step: 5144576	Mean reward: 9.03125
Step: 5152768	Mean reward: 10.09375
Step: 5160960	Mean reward: 9.78125
Step: 5169152	Mean reward: 10.46875
Step: 5177344	Mean reward: 10.03125
Step: 5185536	Mean reward: 9.21875
Step: 5193728	Mean reward: 10.0
Step: 5201920	Mean reward: 11.03125
Step: 5210112	Mean reward: 9.46875
Step: 5218304	Mean reward: 9.625
Step: 5226496	Mean reward: 9.0
Step: 5234688	Mean reward: 9.46875
Step: 5242880	Mean reward: 10.40625
Step: 5251072	Mean reward: 11.8125
Step: 5259264	Mean reward: 9.8125
Step: 5267456	Mean reward: 9.6875
Step: 5275648	Mean reward: 9.125
Step:

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

Step: 9076736	Mean reward: 9.15625
Step: 9084928	Mean reward: 8.25
Step: 9093120	Mean reward: 8.6875
Step: 9101312	Mean reward: 9.6875
Step: 9109504	Mean reward: 9.25
Step: 9117696	Mean reward: 10.0625
Step: 9125888	Mean reward: 10.25
Step: 9134080	Mean reward: 9.09375
Step: 9142272	Mean reward: 9.34375
Step: 9150464	Mean reward: 8.9375
Step: 9158656	Mean reward: 9.46875
Step: 9166848	Mean reward: 8.84375
Step: 9175040	Mean reward: 9.625
Step: 9183232	Mean reward: 9.0
Step: 9191424	Mean reward: 9.9375
Step: 9199616	Mean reward: 8.75
Step: 9207808	Mean reward: 8.71875
Step: 9216000	Mean reward: 8.8125
Step: 9224192	Mean reward: 9.03125
Step: 9232384	Mean reward: 8.5625
Step: 9240576	Mean reward: 8.5
Step: 9248768	Mean reward: 9.1875
Step: 9256960	Mean reward: 9.09375
Step: 9265152	Mean reward: 9.4375
Step: 9273344	Mean reward: 8.78125
Step: 9281536	Mean reward: 10.46875
Step: 9289728	Mean reward: 10.125
Step: 9297920	Mean reward: 8.84375
Step: 9306112	Mean reward: 9.59375
Step: 9314304	

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Step: 9584640	Mean reward: 10.125
Step: 9592832	Mean reward: 9.65625
Step: 9601024	Mean reward: 9.15625


Network definitions. We have defined a policy network for you in advance. It uses the popular `NatureDQN` encoder architecture (see below), while policy and value functions are linear projections from the encodings. There is plenty of opportunity to experiment with architectures, so feel free to do that! Perhaps implement the `Impala` encoder from [this paper](https://arxiv.org/pdf/1802.01561.pdf) (perhaps minus the LSTM).

Below cell can be used for policy evaluation and saves an episode to mp4 for you to view.

In [ ]:
import imageio

# Make evaluation environment
eval_env = make_env(num_envs, env_name = 'starpilot',use_backgrounds=True ,start_level=num_levels, num_levels=num_levels)
obs = eval_env.reset()

frames = []
total_reward = []

# Evaluate policy
policy.eval()
for _ in range(512):

  # Use policy
  action, log_prob, value = policy.act(obs)

  # Take step in environment
  obs, reward, done, info = eval_env.step(action)
  total_reward.append(torch.Tensor(reward))

  # Render environment and store
  frame = (torch.Tensor(eval_env.render(mode='rgb_array'))*255.).byte()
  frames.append(frame)

# Calculate average return
total_reward = torch.stack(total_reward).sum(0).mean(0)
print('Average return:', total_reward)

# Save frames as video
frames = torch.stack(frames).numpy()
imageio.mimsave('vid_stack100.mp4', frames, fps=25)

from google.colab import files
files.download('vid_stack100.mp4')


#
#Step: 10006528	Mean reward: 12.428571701049805
#Completed training!

#Average return: tensor(16.3039)